In [1]:
import os
import sys
import time
import argparse
import json
from tqdm import tqdm

import pandas as pd
import numpy as np
import pykeen
from pykeen.evaluation import rank_based_evaluator
import torch
from scipy import stats

from sheaf_kg.train_sheafE_betae import read_dataset, shuffle_datasets, dataset_to_device, sample_answers
import sheaf_kg.complex_functions as cf
# from sheaf_kg.complex_functions import test_batch

PyKEEN was unable to load dataset openbiolinkf1. Try uninstalling PyKEEN with ``pip uninstall pykeen`` then reinstalling
PyKEEN was unable to load dataset openbiolinkf2. Try uninstalling PyKEEN with ``pip uninstall pykeen`` then reinstalling


In [2]:
dataset = 'FB15k-237'
savename = 'TransE_500epochs_64embdim_MarginRankingLossloss_11seed_20210525-1745'

savedir = f'/home/gebhart/projects/sheaf_kg/data/{dataset}'
complex_dataset_loc = f'/home/gebhart/projects/sheaf_kg/data/{dataset}-betae'

test_query_structures = ['1p','2p','3p','2i','3i','ip','pi']


In [3]:
model = torch.load(os.path.join(savedir, savename, 'trained_model.pkl'))

In [4]:
fun_map = {'1p':cf.L_p_traversal_transE,
           '2p':cf.L_p_traversal_transE,
           '3p':cf.L_p_traversal_transE,
           '2i':cf.L_i_traversal_transE,
           '3i':cf.L_i_traversal_transE,
           'ip':cf.L_ip_traversal_transE,
           'pi':cf.L_pi_traversal_transE
          }

def test_batch(model, test_data, model_inverses=False, test_batch_size=5,
                test_query_structures=test_query_structures,
                ks=[1,3,5,10]):
    with torch.no_grad():
        results = []
        for query_structure in test_query_structures:
            print('Running query : {}'.format(query_structure))
            all_avg_ranks = []
            cnt = 0
            num_test = len(test_data[query_structure]['answers'])
            for qix in tqdm(range(0, num_test, test_batch_size)):
                if num_test - qix == 1:
                    continue
                entities = test_data[query_structure]['entities'][qix:qix+test_batch_size]
                relations = test_data[query_structure]['relations'][qix:qix+test_batch_size]
                if model_inverses:
                    inverses = None
                else:
                    inverses = test_data[query_structure]['inverses'][qix:qix+test_batch_size]
                all_answers = test_data[query_structure]['answers'][qix:qix+test_batch_size]
                targets = torch.arange(model.entity_embeddings(indices=None).shape[0]).to(model.device)
                Q = fun_map[query_structure](model, entities, relations, targets, invs=inverses, p=1)
                answer_lens = np.array([len(a) for a in all_answers])
                max_len = answer_lens.max()
                for l in np.unique(answer_lens):
                    idxs = np.where(answer_lens == l)[0]
                    answers = [all_answers[j] for j in idxs]
                    filter_fix = torch.arange(l)
                    for aix in range(len(idxs)):
                        ranks = rank_based_evaluator.compute_rank_from_scores(Q[idxs[aix],answers[aix]].unsqueeze(1), Q[idxs[aix],:].unsqueeze(0))
                        avg_rank = (torch.sort(ranks['realistic'].cpu(), dim=0)[0] - filter_fix).numpy()
                        all_avg_ranks.append(avg_rank)
            all_avg_ranks = np.concatenate(all_avg_ranks)
            rd = {k: np.mean(all_avg_ranks <= k) for k in ks}
            mrr = np.reciprocal(stats.hmean(all_avg_ranks))
            rd['mrr'] = mrr if isinstance(mrr, float) else mrr[0]
            # rd['mr'] = np.mean(all_avg_ranks)
            results.append(rd)

        df = pd.DataFrame(results, index=test_query_structures)
        return df

In [5]:
datasets = read_dataset(complex_dataset_loc)
datasets = dataset_to_device(shuffle_datasets(datasets), model.device)

In [6]:
extension_df = test_batch(model, datasets['test-easy'], model_inverses=True, test_query_structures=test_query_structures)
extension_df*100

  1%|          | 34/3262 [00:00<00:09, 334.40it/s]

Running query : 1p


  1%|          | 12/977 [00:00<00:08, 116.23it/s]

Running query : 2p


  1%|▏         | 14/974 [00:00<00:07, 131.06it/s]

...

Running query : ip


  4%|▍         | 35/857 [00:00<00:02, 342.93it/s]

Running query : pi


100%|██████████| 857/857 [00:02<00:00, 330.57it/s]


,1,3,5,10,mrr
1p,3.148299,8.359390,11.601865,16.994701,7.656185
2p,0.424479,0.916381,1.308122,1.980778,1.035602
3p,0.262868,0.581311,0.805283,1.239433,0.667380
2i,0.260917,0.710866,1.155046,1.888099,0.806937
3i,0.195366,0.592719,0.954973,1.598686,0.647963
ip,0.377372,0.801914,1.124994,1.726986,0.875956
pi,0.201120,0.528217,0.782871,1.306913,0.623386
